
# Vocabulary Merge — English ASL + Arabic ArSL

This notebook runs **after both** training notebooks have completed.  
It merges the two independently-trained model class maps with the shared vocabulary so the backend knows exactly which model index corresponds to each word in both languages.

### What it produces:

`merged_word_vocabulary.csv` — one row per shared word with columns:

| Column | Description |
|---|---|
| `word_id` | Shared identifier (same across both models) |
| `english` | English gloss (ASL) |
| `arabic` | Arabic gloss (ArSL) |
| `category` | Semantic category |
| `wlasl_class` | WLASL class ID |
| `karsl_class` | KArSL class ID |
| `asl_model_index` | Index emitted by the English model's output layer |
| `arsl_model_index` | Index emitted by the Arabic model's output layer |

### When to re-run:

Re-run this notebook any time you retrain either model (class indices can shift if classes are added/removed).


In [ ]:

# ============================================================
# CELL 1: CONFIGURATION — single place to change
# ============================================================
from pathlib import Path
import pandas as pd
import numpy as np

# ⚠️ Update if your drive letter differs
PROJECT_ROOT = Path(r'M:/Term 10/Grad')

WORDS_ROOT = PROJECT_ROOT / 'SLR Main/Words'
SHARED_DIR = WORDS_ROOT / 'Shared'

# Inputs
SHARED_CSV   = SHARED_DIR / 'shared_word_vocabulary.csv'
ASL_CSV      = WORDS_ROOT / 'ASL Word (English)/asl_word_classes.csv'
ARSL_CSV     = WORDS_ROOT / 'ArSL Word (Arabic)/arsl_word_classes.csv'

# Output
OUTPUT_CSV   = SHARED_DIR / 'merged_word_vocabulary.csv'

# ── Verify inputs exist ──────────────────────────────────────
print('=' * 60)
print('📂 INPUT FILES')
print('=' * 60)
for label, path in [('Shared vocab', SHARED_CSV),
                    ('ASL classes',  ASL_CSV),
                    ('ArSL classes', ARSL_CSV)]:
    status = '✅' if path.exists() else '⚠️  NOT FOUND (train the model first)'
    print(f'{status}  {label}: {path.name}')


In [ ]:

# ============================================================
# CELL 2: LOAD SHARED VOCABULARY
# ============================================================
shared = pd.read_csv(SHARED_CSV)
shared['word_id'] = shared['word_id'].astype(int)

print('=' * 60)
print(f'📖 Shared vocabulary: {len(shared)} words')
print(f'   Columns: {list(shared.columns)}')
print(f'   Categories: {sorted(shared["category"].unique())}')
print()
print(shared.head(8).to_string(index=False))


In [ ]:

# ============================================================
# CELL 3: LOAD ENGLISH (ASL) CLASS MAP
# ============================================================
# asl_word_classes.csv has columns: model_class_index, word_id
# model_class_index = the integer the English model outputs (0…N-1)

if ASL_CSV.exists():
    asl_df = pd.read_csv(ASL_CSV)
    asl_df['word_id'] = asl_df['word_id'].astype(int)
    asl_df = asl_df.rename(columns={'model_class_index': 'asl_model_index'})
    print(f'✅ ASL class map loaded: {len(asl_df)} classes')
    print(f'   word_id range     : {asl_df["word_id"].min()} – {asl_df["word_id"].max()}')
    print(f'   model_index range : {asl_df["asl_model_index"].min()} – {asl_df["asl_model_index"].max()}')
    print(asl_df.head(5).to_string(index=False))
else:
    asl_df = pd.DataFrame(columns=['word_id', 'asl_model_index'])
    print('⚠️  asl_word_classes.csv not found — ASL model index will be NaN in output.')
    print('    Train the English notebook first, then re-run this notebook.')


In [ ]:

# ============================================================
# CELL 4: LOAD ARABIC (ArSL) CLASS MAP
# ============================================================
# arsl_word_classes.csv has columns: model_class_index, word_id

if ARSL_CSV.exists():
    arsl_df = pd.read_csv(ARSL_CSV)
    arsl_df['word_id'] = arsl_df['word_id'].astype(int)
    arsl_df = arsl_df.rename(columns={'model_class_index': 'arsl_model_index'})
    print(f'✅ ArSL class map loaded: {len(arsl_df)} classes')
    print(f'   word_id range     : {arsl_df["word_id"].min()} – {arsl_df["word_id"].max()}')
    print(f'   model_index range : {arsl_df["arsl_model_index"].min()} – {arsl_df["arsl_model_index"].max()}')
    print(arsl_df.head(5).to_string(index=False))
else:
    arsl_df = pd.DataFrame(columns=['word_id', 'arsl_model_index'])
    print('⚠️  arsl_word_classes.csv not found — ArSL model index will be NaN in output.')
    print('    Train the Arabic notebook first, then re-run this notebook.')


In [ ]:

# ============================================================
# CELL 5: MERGE INTO UNIFIED VOCABULARY
# ============================================================
# Merge ASL model indices into shared vocab (left join — keeps all shared words)
merged = shared.copy()

if len(asl_df) > 0:
    merged = merged.merge(asl_df[['word_id', 'asl_model_index']], on='word_id', how='left')
else:
    merged['asl_model_index'] = pd.NA

if len(arsl_df) > 0:
    merged = merged.merge(arsl_df[['word_id', 'arsl_model_index']], on='word_id', how='left')
else:
    merged['arsl_model_index'] = pd.NA

# Use nullable integer type so NaN displays cleanly
merged['asl_model_index']  = merged['asl_model_index'].astype('Int64')
merged['arsl_model_index'] = merged['arsl_model_index'].astype('Int64')

# Reorder columns for readability
col_order = ['word_id', 'english', 'arabic', 'category',
             'wlasl_class', 'karsl_class',
             'asl_model_index', 'arsl_model_index']
for c in col_order:
    if c not in merged.columns:
        merged[c] = pd.NA
merged = merged[col_order]

print('=' * 60)
print('🔀 MERGED VOCABULARY')
print('=' * 60)
print(f'   Total rows             : {len(merged)}')
print(f'   Words with ASL index   : {merged["asl_model_index"].notna().sum()}')
print(f'   Words with ArSL index  : {merged["arsl_model_index"].notna().sum()}')
print(f'   Words in BOTH models   : {(merged["asl_model_index"].notna() & merged["arsl_model_index"].notna()).sum()}')
print()
print(merged.head(10).to_string(index=False))


In [ ]:

# ============================================================
# CELL 6: SEPARATE BACK INTO ENGLISH-ONLY / ARABIC-ONLY VIEWS
# ============================================================
# These are read-only views — the source of truth is merged_word_vocabulary.csv

# English words that have a trained ASL model class
english_vocab = merged[merged['asl_model_index'].notna()].copy()
english_vocab = english_vocab[['word_id', 'english', 'category', 'wlasl_class', 'asl_model_index']]
english_vocab = english_vocab.sort_values('asl_model_index').reset_index(drop=True)

# Arabic words that have a trained ArSL model class
arabic_vocab = merged[merged['arsl_model_index'].notna()].copy()
arabic_vocab = arabic_vocab[['word_id', 'arabic', 'category', 'karsl_class', 'arsl_model_index']]
arabic_vocab = arabic_vocab.sort_values('arsl_model_index').reset_index(drop=True)

print('=' * 60)
print(f'🇺🇸 English (ASL) vocab : {len(english_vocab)} words')
print('=' * 60)
print(english_vocab.head(8).to_string(index=False))
print()
print('=' * 60)
print(f'🇸🇦 Arabic (ArSL) vocab : {len(arabic_vocab)} words')
print('=' * 60)
print(arabic_vocab.head(8).to_string(index=False))



## Notes

| File | Description |
|---|---|
| `merged_word_vocabulary.csv` | Full bilingual vocabulary — **use this in the backend** |
| `english_word_vocabulary.csv` | English-only filtered view (rows with ASL model index) |
| `arabic_word_vocabulary.csv` | Arabic-only filtered view (rows with ArSL model index) |

### Re-train workflow:

```
1. Run ASL Word (English)/ASL_Word_Training.ipynb   → produces asl_word_classes.csv
2. Run ArSL Word (Arabic)/ArSL_Word_Training.ipynb  → produces arsl_word_classes.csv
3. Run THIS notebook                                → produces merged_word_vocabulary.csv
```

### If a model index column is NaN:

The model hasn't been trained yet for those words. Train the missing notebook then re-run this one.

### Adding new words:

1. Add new rows to `shared_word_vocabulary.csv` with unique `word_id`
2. Re-train both models (they will auto-detect the new vocab)
3. Re-run this notebook to regenerate the merged file
